In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.transforms import ToTensor
from pandas import Series
import numpy as np
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA = pd.read_parquet('/content/drive/My Drive/MasterProject/36.parquet')
DATA.head(5)

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,min_fan2_0_avg,max_fan2_0_avg,avg_gpu3_core_temp_max,std_gpu3_core_temp_max,min_gpu3_core_temp_max,max_gpu3_core_temp_max,avg_ps0_input_power_min,std_ps0_input_power_min,min_ps0_input_power_min,max_ps0_input_power_min
0,2020-03-09 12:00:00+00:00,21.636362,0.077138,21.600000,21.799999,29.0,0.0,29.0,29.0,30.0,...,4303.030303,4400.000000,35.4,2.302173,33.0,39.0,272.0,44.384682,220.0,340.0
1,2020-03-09 12:15:00+00:00,21.860000,0.128063,21.600000,22.000000,29.0,0.0,29.0,29.0,30.0,...,4306.666667,4393.548387,35.4,2.302173,33.0,39.0,274.0,43.358967,220.0,340.0
2,2020-03-09 12:30:00+00:00,21.885714,0.098975,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,4305.714286,4397.142857,35.4,2.302173,33.0,39.0,272.0,44.384682,220.0,340.0
3,2020-03-09 12:45:00+00:00,21.918917,0.098194,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,4310.810811,4394.594595,35.8,3.114482,33.0,41.0,270.0,40.620192,220.0,330.0
4,2020-03-09 13:00:00+00:00,21.875553,0.096967,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,4306.666667,4388.888889,35.8,3.114482,33.0,41.0,272.0,44.384682,220.0,340.0


In [ ]:
#The dataset has periods of unavailability of Nagios traces, as described in the dataset paper:  https://www.nature.com/articles/s41597-023-02174-3/figures/3
#We suggest dropping the periods where the labels are largely unavailable and using the dataset either after 1.4.2021 or 1.10.2021.
DATA = DATA[DATA['timestamp'] > '2021-04-01']

### parisa change
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)
## parisa change

In [ ]:
DATA.reset_index(drop=True, inplace = True)
DATA = DATA.fillna(0)
DATA['timestamp'] = pd.to_datetime(DATA['timestamp'])
l = DATA.timestamp.diff() == pd.Timedelta(minutes=15) #time consistency -> measurements should be 15 minutes apart
chunks = []
current_chunk = []
for index, value in enumerate(l):
    current_chunk.append(DATA.iloc[index])
    if not value:
        chunks.append(pd.DataFrame(current_chunk))
        current_chunk = []


if current_chunk:
    chunks.append(pd.DataFrame(current_chunk))

In [ ]:
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change
relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
revelent_len = [len(c) for c in chunks if len(c) >= 20]
DATA2 = pd.concat(relevant)
DATA = DATA2
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change end
##3 parisa change
# relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
# DATA2 = pd.concat(relevant)
# DATA = DATA2
# DATA['value'] = DATA['value'].replace(2,1)
# DATA['value'] = DATA['value'].replace(3,1)
# DATA = DATA.drop(columns=['timestamp'])
# DATA = DATA.astype(float)
##3 parisa change end

In [ ]:
DATA[DATA["value"]>0]

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
102,2021-04-03 02:00:00+00:00,24.822222,0.075686,24.600000,25.000000,33.000000,0.000000,33.0,33.0,33.244444,...,64.0,12.200001,0.000000,12.2,12.2,688.000000,290.513721,520.0,1560.0,1
103,2021-04-03 02:15:00+00:00,24.955556,0.083148,24.799999,25.000000,33.000000,0.000000,33.0,33.0,33.777778,...,73.0,12.200001,0.000000,12.2,12.2,687.111111,276.972700,520.0,1560.0,1
104,2021-04-03 02:30:00+00:00,24.951112,0.085952,24.799999,25.000000,33.000000,0.000000,33.0,33.0,33.800000,...,77.0,12.200001,0.000000,12.2,12.2,683.111111,286.122136,500.0,1520.0,1
107,2021-04-03 03:15:00+00:00,24.800003,0.000000,24.799999,24.799999,33.000000,0.000000,33.0,33.0,33.466667,...,65.0,12.200001,0.000000,12.2,12.2,599.111111,180.071865,520.0,1440.0,1
108,2021-04-03 03:30:00+00:00,24.737779,0.092589,24.600000,24.799999,33.000000,0.000000,33.0,33.0,33.088889,...,59.0,12.200001,0.000000,12.2,12.2,691.111111,270.096462,520.0,1420.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44702,2022-07-11 16:00:00+00:00,22.835555,0.076465,22.799999,23.000000,32.355556,1.267641,31.0,34.0,33.755556,...,73.0,12.200001,0.000000,12.2,12.2,704.000000,186.313952,540.0,1520.0,1
45628,2022-07-21 07:30:00+00:00,22.808889,0.041216,22.799999,23.000000,31.000000,0.000000,31.0,31.0,31.000000,...,61.0,12.206668,0.024944,12.2,12.3,687.111111,217.466444,520.0,1640.0,1
46793,2022-08-02 10:45:00+00:00,22.226670,0.067987,22.200001,22.400000,29.000000,0.000000,29.0,29.0,30.000000,...,31.0,12.364446,0.076465,12.2,12.4,454.666667,99.098156,400.0,700.0,1
46794,2022-08-02 11:00:00+00:00,22.217780,0.056916,22.200001,22.400000,29.000000,0.000000,29.0,29.0,30.000000,...,21.0,12.400001,0.000000,12.4,12.4,405.777778,9.064924,400.0,420.0,1


In [ ]:
DATA3= DATA
DATA=DATA.drop(columns=['timestamp'])
original_dtypes = DATA.dtypes
DATA = DATA.astype(float)

In [ ]:
#### add label of next row
DATA['next_value'] = DATA['value'].shift(-1).fillna(0)

In [ ]:
# df_value_0 = DATA[DATA['value'] == 0.0].drop(columns=['value']).to_numpy()

# # Separate rows where 'value' is 1.0
# df_value_1 = DATA[DATA['value'] == 1.0].drop(columns=['value']).to_numpy()

In [ ]:
### use for delete value column
# DATA=DATA.drop(columns=['value'])
original_dtypes = DATA.dtypes

In [ ]:
### use for delete value column
# scaler = preprocessing.MinMaxScaler()  ### use further
# names = DATA.columns  ##
# d_0 = scaler.fit_transform(df_value_0)
# d_1= scaler.fit_transform(df_value_1)
# df_scale_1 = pd.DataFrame(d_1, columns=names)
# df_scale_0 = pd.DataFrame(d_0, columns=names)

In [ ]:
scaler = preprocessing.MinMaxScaler()  ### use further
names = DATA.columns  ##
d = scaler.fit_transform(DATA)
DATA = pd.DataFrame(d, columns=names)
# ### parisa change                                --- here to change more or add time stamp in train

#timestamp_scaler = preprocessing.MinMaxScaler()
#DATA['timestamp'] = timestamp_scaler.fit_transform(DATA3['timestamp'].astype('int64').values.reshape(-1, 1))
#### parisa change end

In [ ]:
# train_data = DATA[:int(DATA.shape[0]*0.8)]
# test_data = DATA[int(DATA.shape[0]*0.8):]

################# parisa change
train_data = DATA[:]

############### end parisa change

In [ ]:
train_data['value'].value_counts()

,count
value,
0.0,50998
1.0,1311


In [ ]:
# df_value_0 = df_value_0.drop(columns=['value']).to_numpy()
# # train_feat = train_feat.to_numpy()
# df_value_1 = df_value_1.drop(columns=['value']).to_numpy()
# # test_feat = test_feat.to_numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [ ]:
train_data_tensor = torch.tensor(train_data.to_numpy(), dtype=torch.float)

In [ ]:
# test_labels = test_data['value']
# test_labels = test_labels.to_numpy()
# test_labels = torch.tensor(test_labels, dtype=torch.float)
# test_labels

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_window = 20 ## 20

def create_data_seq(data,tw):
    seq = []
    for i in range(len(data)-tw):
        x_seq = data[i:i+tw],
        y_seq = data[i+tw:i+tw+1]  ###در ترین میاد 0 تا 20 رو میده و میگه 21 رو حدس بزن
        seq.append((x_seq,y_seq))
    return seq


In [ ]:
train_seq = create_data_seq(train_data_tensor,train_window)

In [ ]:
train_loader = DataLoader(train_seq, batch_size=1, shuffle=True)

X,y  = next(iter(train_loader))
# print(X)


In [ ]:
############### mine for add h,c
# class encoder(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.lstm1 = torch.nn.LSTM(
#             input_size=352,
#             hidden_size=32,
#             batch_first=True,
#             num_layers=1
#         )
#         self.lstm2 = torch.nn.LSTM(
#           input_size=32,
#           hidden_size=8,
#           num_layers=1,
#           batch_first=True
#         )

#     def forward(self, x):
#         batch_size = x.shape[0]
#         h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)

#         x, (_, _) = self.lstm1(x, (h0, c0))

#         h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
#         x, (_, _) = self.lstm2(x, (h0, c0))
#         return x[:, :, :]

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=1382,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )
        self.h_proj_1_to_2 = torch.nn.Linear(32, 16)
        self.c_proj_1_to_2 = torch.nn.Linear(32, 16)

        self.h_proj_2_to_3 = torch.nn.Linear(16, 8)
        self.c_proj_2_to_3 = torch.nn.Linear(16, 8)

    def forward(self, x):
        batch_size = x.shape[0]
        # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))

        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).to(x.device)
        c0 = torch.zeros(1, batch_size, 32).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h0, c0))

        # Resize h1, c1 to match the input size of lstm2
        h1 = self.h_proj_1_to_2(h1)
        c1 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h1, c1))

        # Resize h2, c2 to match the input size of lstm3
        h2 = self.h_proj_2_to_3(h2)
        c2 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h2, c2))
        return x[:, :, :]

In [ ]:
### mine for add h,c
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=1382,
            batch_first=True,
            num_layers=1
        )

        self.h_proj_1_to_2 = torch.nn.Linear(16, 32)
        self.c_proj_1_to_2 = torch.nn.Linear(16, 32)

        self.h_proj_2_to_3 = torch.nn.Linear(32, 1382)
        self.c_proj_2_to_3 = torch.nn.Linear(32, 1382)

    def forward(self, x):
        batch_size = x.shape[0]

        # # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))


        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        h1 = torch.zeros(1, batch_size, 16).to(x.device)
        c1 = torch.zeros(1, batch_size, 16).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h1, c1))

        # Resize h1, c1 to match the input size of lstm2
        h2 = self.h_proj_1_to_2(h1)
        c2 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h2, c2))

        # Resize h2, c2 to match the input size of lstm3
        h3 = self.h_proj_2_to_3(h2)
        c3 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h3, c3))

        return x

In [ ]:
### mine
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8, 16)
        self.fc2 = torch.nn.Linear(16, 8)
        self.decoder = decoder().to(device)  # Adding the decoder to RUAD

    def forward(self, x):
        x = self.encoder(x)
        # Passing through the fully connected layers   ----- new changes
        # x = self.fc1(x)
        # x = self.fc2(x)
        # Passing the result through the decoder ------- end of new changes
        x = self.decoder(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = RUAD()
model

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(1382, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 1382, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=1382, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=1

In [ ]:
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.03)  ### adam ro bezar va bebin
criterion = torch.nn.MSELoss()

In [ ]:
%%time
def train(output,data):
    model.train()
    for d in data:
        #print(type(d))
        x = d[0][0].to(device)
        y = d[1].to(device)
        optimizer.zero_grad()
        out = model(x)
        # print(out[0][9])
        output.append([out[0][train_window-1]])
        loss = criterion(out[0][train_window-1], y[0][0])
        loss.backward()
        optimizer.step()
    return float(loss) , output







############ results
########### result change layer 8 to 32
# Epoch: 01, Loss: 0.0188
# Epoch: 02, Loss: 0.0273
# Epoch: 03, Loss: 0.0033
# CPU times: user 6min 3s, sys: 3.62 s, total: 6min 6s
# Wall time: 6min 12s

########### result add hidden ayer 32
# Epoch: 01, Loss: 0.0021
# Epoch: 02, Loss: 0.0378
# Epoch: 03, Loss: 0.0069
# CPU times: user 7min 58s, sys: 4.28 s, total: 8min 2s
# Wall time: 8min 11s

########### result add hidden layer 32 to decoder too and add decoder to ruad model with gpu
# Epoch: 01, Loss: 0.0200
# Epoch: 02, Loss: 0.0210
# Epoch: 03, Loss: 0.0231
# CPU times: user 13min 7s, sys: 14.9 s, total: 13min 22s
# Wall time: 13min 5s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu
# Epoch: 01, Loss: 0.0089
# Epoch: 02, Loss: 0.0483
# Epoch: 03, Loss: 0.0035
# CPU times: user 12min 38s, sys: 13.5 s, total: 12min 51s
# Wall time: 12min 34s

########### result first ruad model with gpu and add timestamp column and window 10 and seq
# Epoch: 01, Loss: 0.0107
# Epoch: 02, Loss: 0.0098
# Epoch: 03, Loss: 0.0219
# CPU times: user 11min 17s, sys: 4.6 s, total: 11min 22s
# Wall time: 11min 28s

########### result first ruad model with gpu and add timestamp column and window 10 and remove seq
# Epoch: 01, Loss: 0.0252
# Epoch: 02, Loss: 0.0093
# Epoch: 03, Loss: 0.0188
# CPU times: user 11min 8s, sys: 4.67 s, total: 11min 12s
# Wall time: 11min 19s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , adam optimizer
# Epoch: 01, Loss: 0.0365
# Epoch: 02, Loss: 0.0473
# Epoch: 03, Loss: 0.0536
# CPU times: user 12min 14s, sys: 13.8 s, total: 12min 28s
# Wall time: 12min 12s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer
# Epoch: 01, Loss: 0.0594
# Epoch: 02, Loss: 0.0260
# Epoch: 03, Loss: 0.0311
# CPU times: user 11min 27s, sys: 13.3 s, total: 11min 40s
# Wall time: 11min 24s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 8
# Epoch: 01, Loss: 0.0270
# Epoch: 02, Loss: 0.0267
# Epoch: 03, Loss: 0.0317
# Epoch: 04, Loss: 0.0313
# Epoch: 05, Loss: 0.0615
# CPU times: user 19min 21s, sys: 23.7 s, total: 19min 44s
# Wall time: 19min 18s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 5 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# CPU times: user 20min 12s, sys: 12 s, total: 20min 24s
# Wall time: 20min 39s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# Epoch: 06, Loss: 0.0030
# CPU times: user 24min 16s, sys: 14.37 s, total: 24min 30s
# Wall time: 24min 48s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0042
# Epoch: 02, Loss: 0.0035
# Epoch: 03, Loss: 0.0061
# Epoch: 04, Loss: 0.0053
# Epoch: 05, Loss: 0.0214
# Epoch: 06, Loss: 0.0109
# CPU times: user 26min 13s, sys: 14.2 s, total: 26min 27s
# Wall time: 26min 46s


CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs


In [ ]:
### before run this you need to rerun the model first
output=[]
i=0
for epoch in range(6):
    loss ,output = train(output,train_loader)
    if i!=5:
      output=[]
    i+=1
    print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')

# Epoch: 01, Loss: 0.0128
# Epoch: 02, Loss: 0.0099
# Epoch: 03, Loss: 0.0059
# Epoch: 04, Loss: 0.0236
# Epoch: 05, Loss: 0.0106
# Epoch: 06, Loss: 0.0109

Epoch: 01, Loss: 0.0109
Epoch: 02, Loss: 0.0085
Epoch: 03, Loss: 0.0081
Epoch: 04, Loss: 0.0077
Epoch: 05, Loss: 0.0118
Epoch: 06, Loss: 0.0102


In [ ]:
# def tensor_to_dataframe(tensor, columns, device):
#     numpy_array = tensor.cpu().numpy()  # Convert tensor to NumPy array
#     df = pd.DataFrame(numpy_array, columns=columns)  # Create DataFrame with original column names
#     return df

In [ ]:
### scale ,name
df_list=[]
for l in output:
  ### tensore darim
  numpy_array = l[0].cpu().detach().numpy().reshape(1, -1)
  df=scaler.inverse_transform(numpy_array)
  df_list.append(df)


In [ ]:
df_combined = pd.concat([pd.DataFrame(df, columns=names) for df in df_list], ignore_index=True)
df_combined=df_combined.astype(original_dtypes)

In [ ]:
df_combined.astype(original_dtypes)

In [ ]:
df_combined[df_combined["ambient_avg"]< 0.30]

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value,next_value


In [ ]:
### add time after ass anomaly for not being complicated on timestamp

timestamps = pd.date_range(start=pd.Timestamp('2021-04-02 00:45:00', tz='UTC'), periods=len(df_combined), freq='15T')

# Add timestamp and value columns to the DataFrame
df_combined['timestamp'] = timestamps

In [ ]:
df_combined

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,avg_gpu3_core_temp_max,std_gpu3_core_temp_max,min_gpu3_core_temp_max,max_gpu3_core_temp_max,avg_ps0_input_power_min,std_ps0_input_power_min,min_ps0_input_power_min,max_ps0_input_power_min,next_value,timestamp
0,22.632605,0.082797,22.509747,22.685978,30.431459,0.138977,30.166260,30.585276,30.848211,0.136500,...,44.083305,6.958302,36.521801,53.254215,341.266022,75.682755,257.505951,442.272919,0.017987,2021-04-02 00:45:00+00:00
1,22.635746,0.083260,22.511705,22.689707,30.434170,0.138600,30.169426,30.589201,30.851128,0.136243,...,44.083694,6.961399,36.526020,53.262257,341.346680,75.735840,257.599915,442.455353,0.017967,2021-04-02 01:00:00+00:00
2,22.638121,0.084007,22.515509,22.694950,30.438974,0.139176,30.173515,30.597065,30.856789,0.135928,...,44.093174,6.962402,36.531658,53.275291,341.457275,75.766037,257.665527,442.639343,0.017918,2021-04-02 01:15:00+00:00
3,22.636614,0.083642,22.515266,22.696047,30.436201,0.138888,30.173256,30.596045,30.855408,0.135764,...,44.086926,6.957322,36.531876,53.266701,341.443573,75.814156,257.640503,442.686920,0.019866,2021-04-02 01:30:00+00:00
4,22.636734,0.083944,22.516420,22.699638,30.436991,0.139211,30.175510,30.597576,30.854156,0.135490,...,44.097366,6.955503,36.543251,53.277370,341.487854,75.797501,257.715240,442.693054,0.019795,2021-04-02 01:45:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52284,23.003952,0.093164,22.819515,23.221783,30.791731,0.144282,30.612122,31.044794,31.318443,0.126658,...,44.885368,7.271867,37.101311,54.343174,345.955841,79.826424,259.754578,452.721283,0.021276,2022-09-28 15:45:00+00:00
52285,23.004211,0.093602,22.820475,23.223286,30.793638,0.146546,30.610207,31.046686,31.318924,0.126308,...,44.887062,7.274051,37.096691,54.347919,345.964020,79.884560,259.701660,452.803894,0.021321,2022-09-28 16:00:00+00:00
52286,23.006773,0.094759,22.826530,23.229473,30.801615,0.146848,30.616911,31.055346,31.325949,0.126607,...,44.898525,7.281624,37.102394,54.369370,346.008148,79.850533,259.801544,452.812073,0.021217,2022-09-28 16:15:00+00:00
52287,23.004801,0.095116,22.823126,23.225481,30.801809,0.146473,30.613903,31.053631,31.324492,0.126373,...,44.894108,7.276021,37.099930,54.355370,346.046082,79.836395,259.824860,452.832855,0.021207,2022-09-28 16:30:00+00:00


In [ ]:
df_combined[df_combined["value"]>0.036]

,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,dimm10_temp_std,...,avg_gpu3_core_temp_max,std_gpu3_core_temp_max,min_gpu3_core_temp_max,max_gpu3_core_temp_max,avg_ps0_input_power_min,std_ps0_input_power_min,min_ps0_input_power_min,max_ps0_input_power_min,next_value,timestamp
307,23.378464,0.095814,23.275816,23.550234,31.207857,0.150853,31.007940,31.427395,31.664316,0.131931,...,45.438332,7.252637,37.552368,54.732075,350.723358,80.514565,261.345306,456.141296,0.038237,2021-04-05 05:30:00+00:00
308,23.383062,0.097999,23.272125,23.548088,31.210857,0.151879,31.009834,31.420740,31.661558,0.133439,...,45.474918,7.269116,37.569706,54.784035,350.685242,80.542824,261.231354,456.417297,0.038180,2021-04-05 05:45:00+00:00
309,23.386850,0.097293,23.275335,23.553568,31.210226,0.151477,31.010799,31.422104,31.664772,0.132884,...,45.457912,7.267287,37.555492,54.767910,350.518036,80.544304,260.976349,456.107758,0.038054,2021-04-05 06:00:00+00:00
310,23.380985,0.098139,23.261982,23.539764,31.199844,0.152554,31.000185,31.412922,31.657185,0.132280,...,45.435989,7.250560,37.526802,54.729893,350.236969,80.395683,260.959747,455.892731,0.038061,2021-04-05 06:15:00+00:00
311,23.220192,0.095941,23.116230,23.385471,31.024540,0.151571,30.822611,31.244719,31.464315,0.130024,...,45.163872,7.220460,37.286110,54.398438,348.187988,80.130676,259.284210,452.657928,0.037666,2021-04-05 06:30:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52269,22.715630,0.082943,22.584221,22.818275,30.486738,0.149331,30.287809,30.684849,30.917591,0.138742,...,44.112656,6.909254,36.552959,53.239384,341.958679,76.139145,257.800140,443.650940,0.037283,2022-09-28 12:00:00+00:00
52270,22.712751,0.083807,22.575775,22.817068,30.486948,0.150000,30.287361,30.686705,30.918356,0.138910,...,44.110149,6.918755,36.544907,53.262440,341.924469,76.059319,257.833069,443.549164,0.037170,2022-09-28 12:15:00+00:00
52271,22.714123,0.084475,22.580025,22.817789,30.489939,0.149752,30.287827,30.683483,30.918266,0.138581,...,44.112289,6.921415,36.540100,53.266689,341.966522,75.960007,257.981842,443.334381,0.037084,2022-09-28 12:30:00+00:00
52272,22.712198,0.085466,22.569660,22.814600,30.486933,0.150809,30.283979,30.684166,30.916979,0.139267,...,44.111000,6.930989,36.530800,53.284264,341.915375,75.909180,257.892029,443.199982,0.036971,2022-09-28 12:45:00+00:00


In [ ]:
### save it on google drive
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/MasterProject/'+'create_data_my_model_add_label_5_lr_0.03_inculde_networks'+'.parquet'
df_combined.to_parquet(file_path, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_combined[['value','next_value']]

,value,next_value
0,0.014435,0.017987
1,0.014382,0.017967
2,0.014430,0.017918
3,0.017675,0.019866
4,0.017615,0.019795
...,...,...
52284,0.021303,0.021276
52285,0.021291,0.021321
52286,0.021182,0.021217
52287,0.021018,0.021207


In [ ]:
#0.8765328998395648
#0.8723729179726375    result of change hidden layer from 8 to 32
#0.9065247187071686    result of add one layer 32 between encoder :)
#0.904416749009233     result add hidden layer 32 to decoder too and add decoder to ruad model with gpu (this is pretty good)
#0.8655186444515994    result first ruad model with gpu and add timestamp column and woindow 10 and seq , adam optimizer
#0.8967381497429511    result first ruad model with gpu and add timestamp column and woindow 10 and remove seq , adam optimizer
#0.3060959342747257    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, adam optimizer
#0.34766214604908874   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer
#0.36209899126422795   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 +3 (i forgot to reset)
#0.907213292243243     result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 , add , h,c
#0.9098181020482538    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 6 , add , h,c
#0.9069957536027086    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq, SGD optimizer , epoch 6 , add , h,c